# Retrain EfficientDet-Lite0 Model for the "Modelrailway-Cam"

In this jupyter-notebook, we'll retrain an EfficientDet-Lite object detection model (derived from [EfficientDet](https://ai.googleblog.com/2020/04/efficientdet-towards-scalable-and.html)) using the [TensorFlow Lite Model Maker library](https://www.tensorflow.org/lite/guide/model_maker), and then compile it to run on the [Coral Edge TPU](https://www.coral.ai/products/). All in about 10 minutes on a GPU. Please change runtime type (Laufzeittyp) to "GPU" in the menue.

This notebook retrains the model using images of a modelrailway showing locomotives and waggons. It is an adapted version of the original notebook: [Train a salad detector with TFLite Model Maker - Colaboratory (google.com)](https://colab.research.google.com/github/googlecodelabs/odml-pathways/blob/main/object-detection/codelab2/python/Train_a_salad_detector_with_TFLite_Model_Maker.ipynb)

In [1]:
#@title
# -*- coding: utf-8 -*-

In [2]:
#@title
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [3]:
#Use your google drive for this notebook. Follow messages on scren.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The training data used must be present in the path /content/drive/MyDrive/TrainData


## Import the required packages

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools
!pip install -q tflite-support

     |████████████████████████████████| 642 kB 5.1 MB/s 
     |████████████████████████████████| 55.1 MB 144 kB/s 
     |████████████████████████████████| 3.4 MB 57.8 MB/s 
     |████████████████████████████████| 1.1 MB 62.9 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 120 kB 72.8 MB/s 
     |████████████████████████████████| 6.4 MB 55.0 MB/s 
     |████████████████████████████████| 237 kB 54.6 MB/s 
     |████████████████████████████████| 840 kB 73.9 MB/s 
     |████████████████████████████████| 10.9 MB 29.2 MB/s 
     |████████████████████████████████| 1.2 MB 65.4 MB/s 
     |████████████████████████████████| 596 kB 68.0 MB/s 
     |████████████████████████████████| 87 kB 7.4 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 11.3 MB/s 
     |████████████████████████████████| 352 kB 68.0 MB/s 
     |████████████████████████████████| 48.3 MB 118 kB/s 
     |█████████████

In [5]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

## Load the training data


### Load the training data set by using training.csv



Model Maker requires that we load our dataset using the [`DataLoader`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/DataLoader) API. So in this case, we'll load it from a CSV file that defines the images for training, images for validation, and images for testing. At the start change directory  to the Data directory.




In [6]:
#Load the CSV file from your Google Drive.
%cd drive/MyDrive/TrainData
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('training.csv')

/content/drive/MyDrive/TrainData


## Select the model spec

Model Maker supports the EfficientDet-Lite family of object detection models that are compatible with the Edge TPU. (EfficientDet-Lite is derived from [EfficientDet](https://ai.googleblog.com/2020/04/efficientdet-towards-scalable-and.html), which offers state-of-the-art accuracy in a small model size). There are several model sizes you can choose from:

|| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|-|--------------------|-----------|---------------|----------------------|
|| EfficientDet-Lite0 | 5.7       | 37.4            | 30.4%               |
|| EfficientDet-Lite1 | 7.6       | 56.3            | 34.3%               |
|| EfficientDet-Lite2 | 10.2      | 104.6           | 36.0%               |
|| EfficientDet-Lite3 | 14.4      | 107.6           | 39.4%               |
| <td colspan=4><br><i>* File size of the compiled Edge TPU models. <br/>** Latency measured on a desktop CPU with a Coral USB Accelerator. <br/>*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.</i></td> |

Beware that the Lite2 and Lite3 models do not fit onto the Edge TPU's onboard memory, so you'll see even greater latency when using those, due to the cost of fetching data from the host system memory. Maybe this extra latency is okay for your application, but if it's not and you require the precision of the larger models, then you can [pipeline the model across multiple Edge TPUs](https://coral.ai/docs/edgetpu/pipeline/) (more about this when we compile the model below).

For the modelrailway-cam, we'll use Lite0:

In [7]:
spec = object_detector.EfficientDetLite0Spec()

## Create and train the model

Now we need to create our model according to the model spec, load our dataset into the model, specify training parameters, and begin training. 

Using Model Maker, we accomplished all of that with [`create()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/create):

In [8]:
model = object_detector.create(train_data=train_data, 
                               model_spec=spec, 
                               validation_data=validation_data, 
                               epochs=40, 
                               batch_size=8, 
                               train_whole_model=True)

Epoch 1/40
46/46 [==============================] - 55s 305ms/step - det_loss: 1.3742 - cls_loss: 0.8220 - box_loss: 0.0110 - reg_l2_loss: 0.0630 - loss: 1.4372 - learning_rate: 0.0090 - gradient_norm: 2.4682 - val_det_loss: 0.9057 - val_cls_loss: 0.6127 - val_box_loss: 0.0059 - val_reg_l2_loss: 0.0630 - val_loss: 0.9688
Epoch 2/40
46/46 [==============================] - 11s 246ms/step - det_loss: 0.6453 - cls_loss: 0.3244 - box_loss: 0.0064 - reg_l2_loss: 0.0631 - loss: 0.7084 - learning_rate: 0.0100 - gradient_norm: 2.9727 - val_det_loss: 0.3519 - val_cls_loss: 0.2045 - val_box_loss: 0.0029 - val_reg_l2_loss: 0.0631 - val_loss: 0.4150
Epoch 3/40
46/46 [==============================] - 12s 251ms/step - det_loss: 0.4469 - cls_loss: 0.2539 - box_loss: 0.0039 - reg_l2_loss: 0.0631 - loss: 0.5100 - learning_rate: 0.0099 - gradient_norm: 2.7947 - val_det_loss: 0.3168 - val_cls_loss: 0.1817 - val_box_loss: 0.0027 - val_reg_l2_loss: 0.0631 - val_loss: 0.3799
Epoch 4/40
46/46 [=============

## Evaluate the model

Now we'll use the test dataset to evaluate how well the model performs with data it has never seen before.

The [`evaluate()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/ObjectDetector#evaluate) method provides output in the style of [COCO evaluation metrics](https://cocodataset.org/#detection-eval):

In [9]:
model.evaluate(test_data)

1/1 [==============================] - 8s 8s/step



{'AP': 0.86008596,
 'AP50': 1.0,
 'AP75': 1.0,
 'AP_/:Dampflok': 0.82570326,
 'AP_/:Diesellok': 0.8944686,
 'APl': -1.0,
 'APm': 0.86020887,
 'APs': -1.0,
 'ARl': -1.0,
 'ARm': 0.9,
 'ARmax1': 0.8730769,
 'ARmax10': 0.9,
 'ARmax100': 0.9,
 'ARs': -1.0}

Because the default batch size for [EfficientDetLite models](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/EfficientDetSpec) is 64, this needs only 1 step to go through all  images in the test set. You can also specify the `batch_size` argument when you call [`evaluate()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/ObjectDetector#evaluate).

## Export to TensorFlow Lite

Next, we'll export the model to the TensorFlow Lite format. By default, the [`export()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/ObjectDetector#export) method performs [full integer post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization#full_integer_quantization), which is exactly what we need for compatibility with the Edge TPU. (Model Maker uses the same dataset we gave to our model spec as a representative dataset, which is required for full-int quantization.)

We just need to specify the export directory and format. By default, it exports to TF Lite, but we also want a labels file, so we declare both:

In [10]:
TFLITE_FILENAME = 'smrc_model.tflite'
LABELS_FILENAME = 'railwayLabels.txt'

In [11]:
model.export(export_dir='.', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

### Evaluate the TF Lite model

Exporting the model to TensorFlow Lite can affect the model accuracy, due to the reduced numerical precision from quantization and because the original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TF Lite model uses global NMS, which is faster but less accurate.

Therefore you should always evaluate the exported TF Lite model and be sure it still meets your requirements:

In [12]:
model.evaluate_tflite(TFLITE_FILENAME, test_data)

52/52 [==============================] - 122s 2s/step



{'AP': 0.8533238,
 'AP50': 1.0,
 'AP75': 1.0,
 'AP_/:Dampflok': 0.815861,
 'AP_/:Diesellok': 0.8907867,
 'APl': -1.0,
 'APm': 0.8533238,
 'APs': -1.0,
 'ARl': -1.0,
 'ARm': 0.8769231,
 'ARmax1': 0.8769231,
 'ARmax10': 0.8769231,
 'ARmax100': 0.8769231,
 'ARs': -1.0}

## Compile for the Edge TPU


First we need to download the Edge TPU Compiler:

In [13]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  87482      0 --:--:-- --:--:-- --:--:-- 87482
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,722 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/re

Before compiling the `.tflite` file for the Edge TPU, it's important to consider whether your model will fit into the Edge TPU memory. 

The Edge TPU has approximately 8 MB of SRAM for [caching model paramaters](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching), so any model close to or over 8 MB will not fit onto the Edge TPU memory. That means the inference times are longer, because some model parameters must be fetched from the host system memory.

One way to elimiate the extra latency is to use [model pipelining](https://coral.ai/docs/edgetpu/pipeline/), which splits the model into segments that can run on separate Edge TPUs in series. This can significantly reduce the latency for big models.

The following table provides recommendations for the number of Edge TPUs to use with each EfficientDet-Lite model.

| Model architecture | Minimum TPUs | Recommended TPUs
|--------------------|-------|-------|
| EfficientDet-Lite0 | 1     | 1     |
| EfficientDet-Lite1 | 1     | 1     |
| EfficientDet-Lite2 | 1     | 2     |
| EfficientDet-Lite3 | 2     | 2     |
| EfficientDet-Lite4 | 2     | 3     |

If you need extra Edge TPUs for your model, then update `NUMBER_OF_TPUS` here:

In [14]:
NUMBER_OF_TPUS =  1

!edgetpu_compiler --min_runtime_version 13 $TFLITE_FILENAME -d --num_segments=$NUMBER_OF_TPUS 

Edge TPU Compiler version 16.0.384591198
Searching for valid delegate with step 1
Try to compile segment with 267 ops
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 2748 ms.

Input model: smrc_model.tflite
Input size: 4.24MiB
Output model: smrc_model_edgetpu.tflite
Output size: 5.57MiB
On-chip memory used for caching model parameters: 4.21MiB
On-chip memory remaining for caching model parameters: 3.29MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 267
Operation log: smrc_model_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 264
Number of operations that will run on CPU: 3

**Beware when using multiple segments:** The Edge TPU Comiler divides the model such that all segments have roughly equal amounts of parameter data, but that does not mean all segments have the same latency. Especially when dividing an SSD model such as EfficientDet, this results in a latency-imbalance between segments, because SSD models have a large post-processing op that actually executes on the CPU, not on the Edge TPU. So although segmenting your model this way is better than running the whole model on just one Edge TPU, we recommend that you segment the EfficientDet-Lite model using our [profiling-based partitioner tool](https://github.com/google-coral/libcoral/tree/master/coral/tools/partitioner#profiling-based-partitioner-for-the-edge-tpu-compiler), which measures each segment's latency on the Edge TPU and then iteratively adjusts the segmentation sizes to provide balanced latency between all segments.

## Download the files

In [15]:
from google.colab import files

files.download(TFLITE_FILENAME)
files.download(TFLITE_FILENAME.replace('.tflite', '_edgetpu.tflite'))
files.download(LABELS_FILENAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## More resources

* For more information about the Model Maker library used in this tutorial, see the [TensorFlow Lite Model Maker guide](https://www.tensorflow.org/lite/guide/model_maker) and [API reference](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker).

* For other transfer learning tutorials that are compatible with the Edge TPU, see the [Colab tutorials for Coral](https://github.com/google-coral/tutorials#colab-tutorials-for-coral).

* You can also find more examples that show how to run inference on the Edge TPU at [coral.ai/examples](https://coral.ai/examples/#code-examples/).